## **Ukázky: Multi-AI agenti pro rezervaci hotelu**

V dnešním uspěchaném světě plánování služební cesty zahrnuje víc než jen rezervaci letu a hotelového pokoje. Vyžaduje úroveň koordinace a efektivity, kterou může být obtížné dosáhnout. Právě zde přicházejí na scénu Multi-AI agenti, kteří revolučním způsobem mění způsob, jakým spravujeme naše cestovní potřeby.

Představte si tým inteligentních agentů, kteří jsou vám k dispozici a společně se starají o každý aspekt vaší cesty s přesností a lehkostí. Díky naší pokročilé AI technologii jsme vytvořili specializované agenty pro rezervace služeb a plánování itineráře, což zajišťuje bezproblémový a bezstresový cestovní zážitek.

Toto je základní scénář. Při plánování služební cesty je potřeba konzultovat s cestovním agentem informace o letenkách, hotelech atd. Prostřednictvím AI agentů můžeme vytvořit agenty pro rezervace služeb a agenty pro plánování itineráře, kteří spolupracují a zvyšují úroveň inteligence.


# Inicializace služby Azure AI Agent a získání konfiguračních informací z **.env**

### **.env** 

Vytvořte soubor .env 

Soubor **.env** obsahuje připojovací řetězec služby Azure AI Agent, model používaný AOAI a odpovídající API služby Google API Search, ENDPOINT atd.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Název nasazení modelu služby Azure AI Agent"

[**NOTE**] Budete potřebovat model s limitem 100 000 tokenů za minutu (Rate Limit) a limitem 600 požadavků za minutu (Request per minute).

  Model můžete získat v Azure AI Foundry - Model and Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Připojovací řetězec projektu služby Azure AI Agent"

  Připojovací řetězec projektu najdete v přehledu projektu na obrazovce AI ​​Foundry Portal.

- **SERPAPI_SEARCH_API_KEY** = "Váš SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Váš SERPAPI Search Endpoint"

Pro získání názvu nasazení modelu a připojovacího řetězce projektu služby Azure AI Agent je nutné vytvořit službu Azure AI Agent. Doporučuje se použít [tuto šablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) pro přímé vytvoření. (***Poznámka:*** Služba Azure AI Agent je aktuálně nastavena v omezených regionech. Doporučuje se odkazovat na [tento odkaz](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) pro nastavení regionu.)

Agent potřebuje přístup k SERPAPI. Doporučuje se zaregistrovat pomocí [tohoto odkazu](https://serpapi.com/searches). Po registraci můžete získat jedinečný API KEY a ENDPOINT.


# Přihlášení do Azure

Nyní se musíte přihlásit do Azure. Otevřete terminál ve VScode a spusťte příkaz `az login`.


# Nastavení

Pro spuštění tohoto notebooku budete potřebovat nainstalovat následující knihovny. Zde je seznam požadovaných knihoven a odpovídající příkazy pro instalaci pomocí pip:

azure-identity: Pro autentizaci Azure.
requests: Pro provádění HTTP požadavků.
semantic-kernel: Pro framework semantic kernel (pokud se jedná o vlastní nebo specifickou knihovnu, možná ji budete muset nainstalovat z konkrétního zdroje nebo repozitáře).


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Vysvětlení: 
import asyncio: Tímto se importuje modul asyncio, který poskytuje podporu pro asynchronní programování v Pythonu. Umožňuje psát souběžný kód pomocí syntaxe async a await.
from typing: Annotated: Tímto se importuje typ Annotated z modulu typing. Annotated se používá k přidání metadata k typovým anotacím, což může být užitečné pro různé účely, jako je validace, dokumentace nebo nástroje.


In [ ]:
import asyncio,os
from typing import Annotated

# Vysvětlení:
Použitím from dotenv import load_dotenv a load_dotenv() můžete snadno spravovat nastavení konfigurace a citlivé informace (jako API klíče a URL adresy databází) v souboru .env, čímž je oddělíte od zdrojového kódu. To zvyšuje bezpečnost vaší aplikace a usnadňuje její konfiguraci.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Vysvětlení:

Import příkazu: from azure.identity.aio import DefaultAzureCredential: Tento příkaz importuje třídu DefaultAzureCredential z modulu azure.identity.aio. Část názvu modulu "aio" naznačuje, že je navržen pro asynchronní operace.

Účel DefaultAzureCredential: Třída DefaultAzureCredential je součástí Azure SDK pro Python. Poskytuje výchozí způsob autentizace s Azure službami. Pokouší se autentizovat pomocí několika metod ve specifickém pořadí, jako jsou proměnné prostředí, spravovaná identita a přihlašovací údaje Azure CLI.

Asynchronní operace: Modul "aio" naznačuje, že třída DefaultAzureCredential podporuje asynchronní operace. To znamená, že ji můžete použít s asyncio pro provádění neblokujících autentizačních požadavků.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Vysvětlení:
Importuje různé moduly a třídy z balíčku semantic_kernel. Zde je rozbor jednotlivých importů:

AgentGroupChat z semantic_kernel.agents: Tato třída zajišťuje funkce související se skupinovým chatem pro AI agenty. AzureAIAgent a AzureAIAgentSettings z semantic_kernel.agents.azure_ai

AzureAIAgent: Tato třída slouží k vytváření a správě AI agentů, kteří využívají služby Azure AI.

AzureAIAgentSettings: Tato třída slouží ke konfiguraci nastavení pro AzureAIAgent. TerminationStrategy z semantic_kernel.agents.strategies.termination.termination_strategy:

Tato třída definuje strategie pro ukončení činnosti AI agentů za určitých podmínek. ChatMessageContent z semantic_kernel.contents.chat_message_content:

Tato třída slouží ke zpracování obsahu chatových zpráv.
AuthorRole z semantic_kernel.contents.utils.author_role:

Tato třída definuje různé role autorů v kontextu chatových zpráv.

kernel_function z semantic_kernel.functions.kernel_function_decorator: Tento dekorátor se používá k definování kernel funkcí, což jsou funkce, které lze spustit v rámci frameworku semantic kernel.
Tyto importy nastavují potřebné komponenty pro vytváření a správu AI agentů, kteří mohou komunikovat ve skupinovém chatu, například pro úkoly jako rezervace hotelů nebo podobné aktivity.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Vysvětlení:
Dále importujeme třídu CodeInterpreterTool z modulu azure.ai.projects.models.

CodeInterpreterTool: Tato třída je součástí Azure AI SDK a slouží k interpretaci a provádění kódu v rámci AI projektů. Poskytuje funkce pro spuštění úryvků kódu, analýzu kódu nebo integraci provádění kódu do AI pracovních postupů. Tento import nastavuje potřebnou komponentu pro využití CodeInterpreterTool ve vašem projektu, což může být užitečné pro úkoly zahrnující dynamickou interpretaci a provádění kódu.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Vysvětlení:  
Třída ApprovalTerminationStrategy poskytuje specifickou strategii pro ukončení činnosti AI agenta. Agent ukončí svou činnost, pokud poslední zpráva v jeho historii interakcí obsahuje slovo „uloženo“. To může být užitečné ve scénářích, kdy je úkol agenta považován za dokončený, jakmile obdrží potvrzení, že něco bylo „uloženo“. Definujte metodu interakce. Po uložení plánu rezervace může být činnost ukončena při přijetí signálu „uloženo“.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Vysvětlení:

Řádek kódu inicializuje objekt AzureAIAgentSettings s výchozím nebo předem definovaným nastavením pomocí metody create(). Tento objekt nastavení (ai_agent_settings) může být následně použit k konfiguraci a správě instance AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Vysvětlení:
Importováním knihovny requests můžete snadno provádět HTTP požadavky a komunikovat s webovými službami ve vašem Python kódu.


In [ ]:
import requests

# Vysvětlení:
Toto je proměnná, která ukládá API klíč pro přístup k API službě SERP (Search Engine Results Page). API klíč je jedinečný identifikátor používaný k ověření požadavků spojených s vaším účtem.

'GOOGLE_SEARCH_API_KEY': Toto je zástupný řetězec. Musíte nahradit ''GOOGLE_SEARCH_API_KEY'' vaším skutečným API klíčem pro SERP.

Účel: Účelem tohoto řádku je uložit API klíč do proměnné, aby mohl být použit k ověření požadavků na API službu SERP. API klíč je nezbytný pro přístup ke službě a provádění vyhledávání.

Jak získat API klíč pro SERP: Chcete-li získat API klíč pro SERP, postupujte podle těchto obecných kroků na https://serpapi.com (konkrétní kroky se mohou lišit v závislosti na používané službě SERP API):

Vyberte si službu SERP API: Existuje několik dostupných služeb SERP API, jako například SerpAPI, Google Custom Search JSON API a další. Vyberte si tu, která nejlépe vyhovuje vašim potřebám.

Zaregistrujte si účet:

Přejděte na web vybrané služby SERP API https://www.serpapi.com a zaregistrujte si účet. Možná budete muset poskytnout základní informace a ověřit svou e-mailovou adresu.

Vytvořte API klíč:

Po registraci se přihlaste ke svému účtu a přejděte do sekce API nebo na ovládací panel. Hledejte možnost vytvořit nebo vygenerovat nový API klíč.
Zkopírujte API klíč:

Jakmile je API klíč vygenerován, zkopírujte jej. Tento klíč bude použit k ověření vašich požadavků na službu SERP API.
Nahraďte zástupný řetězec:

Nahraďte zástupný řetězec ve vašem souboru .env.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Vysvětlení:
Třída BookingPlugin poskytuje metody pro rezervaci hotelů a letů pomocí Google Search API od Serpapi.com. Sestavuje potřebné parametry, odesílá požadavky na API a zpracovává odpovědi, aby vrátila relevantní informace o rezervacích. API klíč (SERPAPI_SEARCH_API_KEY) a endpoint (SERPAPI_SEARCH_ENDPOINT) se používají k autentizaci a odesílání požadavků na Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Vysvětlení:
Třída SavePlugin poskytuje metodu saving_plan pro ukládání plánů cest pomocí služeb Azure AI. Nastavuje Azure přihlašovací údaje, vytváří AI agenta, zpracovává vstupy uživatele pro vytvoření a uložení obsahu plánu cesty a zajišťuje operace ukládání souborů a jejich vyčištění. Metoda vrací "Uloženo" po úspěšném dokončení.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Vysvětlení:
Tento kód nastavuje agenty Azure AI pro zajištění rezervace letů a hotelů a ukládání plánů cest na základě vstupů od uživatelů. Používá přihlašovací údaje Azure k vytvoření a konfiguraci agentů, zpracovává vstupy uživatelů prostřednictvím skupinového chatu a zajišťuje správné vyčištění po dokončení úkolů. Agenti používají specifické pluginy (BookingPlugin a SavePlugin) k provádění svých příslušných úkolů.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby AI pro překlady [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Nenese odpovědnost za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
